<a href="https://colab.research.google.com/github/RodrigoSalles/Big-Data-and-Cloud-Computing---Colab/blob/master/Zipf's_law.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example data processing - Zipf's law

**[Big Data and Cloud Computing](https://www.dcc.fc.up.pt/~edrdo/aulas/bdcc), Eduardo R. B. Marques, DCC/FCUP**


## Zipf's law

In a corpus (e.g., list of documents) of a language (e.g., Portuguese), the frequency of a word tends to be inversely proportional to its rank in the global list of words sorted by decreasing frequency. 

This trait is captured by [Zipf's law](https://en.wikipedia.org/wiki/Zipf%27s_law), formulated by American linguist [George Kingsley Zipf](https://en.wikipedia.org/wiki/George_Kingsley_Zipf). 

Let $n$ be the number words in a corpus. For a fixed exponent $s > 0$ (usually close to $1$), Zipf's law estimates that the frequency of the $k$-th most frequent word is approximated by

$$
f(k) = \frac{(1/k^s)}{ \sum_{i=1}^n (1 / i^s)}
$$

In this notebook we analyze if the English language conforms to Zipf's law, taking as corpus the "[Complete Works of William Shakespeare](https://www.gutenberg.org/ebooks/100)", available from the [Gutenberg project](https://www.gutenberg.org/).


## Getting the data

In [0]:
root = '/tmp'
data_file = root + '/shakespeare.txt'
!test -f $root/shakespeare.txt || curl https://www.gutenberg.org/files/100/100-0.txt -o $data_file
!ls -l $data_file
!head -10 $data_file

## Calculating word counts

In [0]:
def word_counts(file):
  import re
  with open(file,'r') as f:
    frequency = {}
    data = f.readlines()
    for line in data:
      # print(line)
      line = re.sub('[ 0-9\"\‘\*\.\_\+\&\/\%\-\$\#\'\)\(\[\[\],.!?;:\t\n"]+', ' ', line.strip()).upper()
      for word in line.split():
        # print('W '+ word)
        count = frequency.get(word, 0)
        frequency[word] = count + 1
    return frequency

wf = word_counts(data_file)

n = len(wf)

sorted_wf = sorted(wf.items(),key=lambda item: item[1],reverse=True)

print("Top 10 words:")
for i, v in enumerate(sorted_wf[:10]):
  print('%d: "%s" - %d' % (i+1, v[0], v[1]))


## Calculate Zipf's law estimate





In [0]:
import numpy as np
import pandas as pd
# Calculate the Zipf values for a given frequency array (sorted in decreasing order)
# and exponent.

def zipf(n, s):
   rank = np.arange(1, n+1)
   zipf = (1/rank**s)/(np.sum(1/(rank**s)))
   return zipf

# Words
words = [v[0] for v in sorted_wf]

# Counts in a numpy array
counts = np.array([v[1] for v in sorted_wf])

# Total number of words
N = np.sum(counts)

# Unique words
n = len(counts)

# Frequencies
freq = counts / N

# Exponent
s = 1

# Get zipf estimate
zipf = zipf(n,s)



## Comparing Zipf's estimate with the actual data

In [0]:
# Organize data as a panda data frame
data = pd.DataFrame(data={
     'rank': np.arange(1,n+1),
     'word': words, 
     'freq': freq, 
     'zipf': zipf,
     'count': counts,
     'zipf_count_estimate': zipf * N
})
data.head(20)

## Draw a (log-log) plot 

In [0]:
ax = data[['rank','count','zipf_count_estimate']].plot(x='rank')

ax.set( title='s=%5.2f' % s, xscale="log", yscale="log")

## Exercises

1. Perform a similar analysis considering the Portuguese language and [Os Lusíadas](https://www.gutenberg.org/ebooks/27236).

2. Zipf's law can be approximated by a simpler expression:

   $$f(k) = A / k^s$$
  
  where $s$ is the exponent as before and $A$ is a "normalization" constant. A common value for $A$ is $0.1$. Implement this alternative.

